In [5]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.14 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.5/107.7 GB disk)


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os# Zip dosyasının yolu
#zip_path = '/content/drive/MyDrive/GokmenUAV/Software/computer_vision/datasets/fixed-wing-yolo-data/fixed-wing-update.zip'

# Zip dosyasını çıkartma yolu
#unzip_location = '/content/datasets'

# Zip dosyasını çıkartma komutu
#!unzip -q "$zip_path" -d "$unzip_location"

In [ ]:
from PIL import Image, ImageFile
import os

ImageFile.LOAD_TRUNCATED_IMAGES = True

def resize_and_pad(img_path, target_size=(640, 640)):
    img = Image.open(img_path)

    # Öncelikle orijinal en-boy oranını koruyarak genişliği 640 yapacak şekilde ölçeklendir.
    img_ratio = img.width / img.height
    target_ratio = target_size[0] / target_size[1]

    if img_ratio >= target_ratio:
        # Genişlik, yükseklikten daha büyük veya eşitse, genişliği hedefe ölçeklendir.
        new_height = int(target_size[0] / img_ratio)
        img_resized = img.resize((target_size[0], new_height), Image.ANTIALIAS)
    else:
        # Yükseklik, genişlikten büyükse, yüksekliği hedefe ölçeklendir.
        new_width = int(target_size[1] * img_ratio)
        img_resized = img.resize((new_width, target_size[1]), Image.ANTIALIAS)

    # Oluşan yeni görüntüyü hedef boyuta getirmek için padding ekleyin.
    img_padded = Image.new("RGB", target_size, (255, 255, 255))  # Beyaz arka planla yeni bir görüntü oluştur.
    # Yeni görüntüyü ortalamak için padding hesaplama
    padding_left = (target_size[0] - img_resized.width) // 2
    padding_top = (target_size[1] - img_resized.height) // 2
    img_padded.paste(img_resized, (padding_left, padding_top))

    return img_padded

def process_images(source_folder, target_folder, target_size=(640, 640)):
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(source_folder, filename)
            img_padded = resize_and_pad(img_path, target_size)

            save_path = os.path.join(target_folder, filename)
            img_padded.save(save_path)
            print(f"Processed and saved: {save_path}")

# Görüntülerin bulunduğu klasör
source_folder = '/content/datasets/fixed-wing-update/images/train'
# İşlenmiş görüntülerin kaydedileceği klasör
target_folder = '/content/datasets/fixed-wing-update/images/train_rz'

val_folder = '/content/datasets/fixed-wing-update/images/val'
# İşlenmiş görüntülerin kaydedileceği klasör
val_target = '/content/datasets/fixed-wing-update/images/val_rz'
# İşlemi başlat
process_images(source_folder, target_folder)
process_images(val_folder, val_target)

In [ ]:

!pip install wandb
import wandb
!wandb login

In [ ]:
!yolo train --epochs 3 --data gokmen.yaml --weights yolov8n.pt epochs=3 imgsz=640 --project project --entity your_entity
